In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from preprocessing import converter_para_cinza, aplicar_filtro_gaussiano, detectar_bordas_canny
from segmentation import segmentar_objeto_com_flood_fill, filtrar_contornos_borda, encontrar_contornos, desenhar_contornos
from geometric import calcular_area, calcular_perimetro, calcular_circularidade, calcular_aspect_ratio
from features import extrair_caracteristicas_forma, extrair_lbp, extrair_glcm, extrair_hog

caminho_imagem = '../../dataset_test/Banana/1.jpg'

img_original = cv2.imread(caminho_imagem)
img_rgb = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
img_cinza = converter_para_cinza(img_original)
img_suavizada = aplicar_filtro_gaussiano(img_cinza)
img_bordas_canny = detectar_bordas_canny(img_suavizada)

mascara_segmentada = segmentar_objeto_com_flood_fill(img_suavizada)
contornos = encontrar_contornos(mascara_segmentada)
altura_img, largura_img = img_cinza.shape
contornos_filtrados = filtrar_contornos_borda(contornos, largura_img, altura_img)
mascara_final = np.zeros_like(img_cinza)
contours = cv2.drawContours(mascara_final, contornos_filtrados, -1, (255), -1)
img_com_contornos = desenhar_contornos(img_rgb, contornos_filtrados, cor=(0, 255, 0), espessura=2)

if contornos_filtrados:
    contorno_principal = max(contornos_filtrados, key=cv2.contourArea)
else:
    contorno_principal = None


metricas_geo = []
if contorno_principal is not None:
    area = calcular_area(contorno_principal)
    perimetro = calcular_perimetro(contorno_principal)
    circularidade = calcular_circularidade(contorno_principal)
    aspect_ratio = calcular_aspect_ratio(contorno_principal)
    metricas_geo = [area, perimetro, circularidade, aspect_ratio]

vetor_hog, img_visual_hog = extrair_hog(img_cinza) 
img_visual_lbp, hist_lbp = extrair_lbp(img_cinza)
metricas_glcm = extrair_glcm(img_cinza)

def ensure_flatten(x) -> np.ndarray:
    arr = np.array(x)
    if arr.ndim == 0:
        return np.array([arr])
    elif arr.ndim > 1:
        return arr.flatten()
    return arr

processed_pdi = [img_original, img_rgb, img_cinza, img_suavizada, img_bordas_canny, mascara_segmentada, contornos, altura_img, largura_img, contornos_filtrados, mascara_final, contours, img_com_contornos, metricas_geo, vetor_hog, img_visual_hog, img_visual_lbp, hist_lbp, metricas_glcm]
processed_pdi = [ensure_flatten(i) for i in processed_pdi]

data = np.concatenate(processed_pdi)
pd.DataFrame([data], columns=[f'feat_{i}' for i in range(len(data))])

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_862760,feat_862761,feat_862762,feat_862763,feat_862764,feat_862765,feat_862766,feat_862767,feat_862768,feat_862769
0,255,255,255,255,255,255,255,255,255,255,...,0.029786,0.016791,0.032429,0.042543,0.076006,0.024441,0.038013,0.661818,0.054466,"{'contrast': 306.49704204705466, 'dissimilarit..."
